In [9]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from time import sleep
import pandas as pd
import os


############################################################################################################################################################################

def set_review_order_to_recent():
    max_retries = 3
    retries = 0
    while retries < max_retries:
        try:
            # 새로 고침
            driver.refresh()
            time.sleep(3)
            # 리뷰 정렬 드롭다운 클릭
            driver.execute_script("document.querySelectorAll('#main > section.section.mb-20 > div.section-header._1xvmu764._1xvmu765 > div > div._10fm75h2 > a').forEach(a => a.click())")
            time.sleep(2)
            # '최근 등록순' 클릭
            driver.execute_script("document.querySelectorAll('#main > section.section.mb-20 > div.section-header._1xvmu764._1xvmu765 > div > div._10fm75h2 > div > label:nth-child(2) > span').forEach(a => a.click())")
            time.sleep(4)

            # 정렬이 변경되었는지 확인
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            sort_indicator = soup.select_one('#main > section.section.mb-20 > div.section-header._1xvmu764._1xvmu765 > div > div._10fm75h2 > div > label._10fm75h1._1xvmu767 > span')
            if '최근' in sort_indicator.text:
                print("리뷰가 '최근 등록순'으로 변경되었습니다.")
                break
        except Exception as e:
            print("정렬 변경 중 오류 발생:", e)
        
        retries += 1
        if retries == max_retries:
            print("최대 재시도 횟수 도달. 정렬 변경 실패.")
            break
        else:
            print(f"재시도 {retries}/{max_retries}")
            time.sleep(2)






# 초기 설정 및 URL 열기
url = 'https://app.catchtable.co.kr/'
id = '01076300106'
pw = '=Cherrycow910'

# 브라우저 열기
driver = webdriver.Chrome()
driver.get(url)

# 페이지 로드 대기
time.sleep(5)

wait = WebDriverWait(driver, 10)

# 팝업 정리
driver.execute_script("document.querySelectorAll('#dock > div > nav > a:nth-child(5)').forEach(a => a.click())")           
driver.execute_script("document.querySelectorAll('._1qn1ik44._1ltqxco1e').forEach(a => a.click())")
time.sleep(2)
driver.execute_script("document.querySelectorAll('#main > section > div.__inner > div > div > div._1w07br45 > button:nth-child(3)').forEach(a => a.click())")

# 로그인 절차
wait.until(EC.presence_of_element_located((By.ID, 'login-id'))).send_keys(id)
wait.until(EC.presence_of_element_located((By.ID, 'login-pw'))).send_keys(pw)
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > div > button'))).click()

# 로그인 후 팝업 닫기
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#popup > div > div > div > div > div._1qn1ik43 > button._1qn1ik44._1ltqxco1u'))).click()

# 내 주변 들어가기
time.sleep(3)
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > section:nth-child(6) > div > div.section-body > div > div > div > a.icon'))).click()

# 필터 클릭
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > div.search-header.__autohide > div:nth-child(2) > div > div.speb4g1 > button'))).click()
time.sleep(2)

# 서울에서 특정 지역 (예: 강남) 필터링
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.fldt7j1 > button:nth-child(2)'))).click()

#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > main > div:nth-child(1) > section > div.fldt7j0 > div.fldt7j2 > button.FilterChip_container__1mb4yfk0.app_typo_body_body1__jg3o1w1d.FilterChip_container_isFocused_true__1mb4yfk2
# select문 마지막 child(2) 이부분만 수정해주면 원하는 지역을 찍어줌 괄호안 숫자만 바꾸면 됨
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > main > div:nth-child(1) > section > div.fldt7j0 > div.fldt7j2 > button:nth-child(9) > span'))).click()
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#overlay > div.BottomSheet_bottomSheetContent__1tetrf80.BottomSheet_bottomSheetContent_hasMaxWidth_true__1tetrf81 > div > footer > div.j4ixu30 > div > button'))).click()
time.sleep(2)
driver.execute_script("document.querySelectorAll('#search-result-header > div._10fm75h2 > button').forEach(a => a.click())")
time.sleep(2)
driver.execute_script("document.querySelectorAll('#search-result-header > div._10fm75h2 > div > label:nth-child(3)').forEach(a => a.click())")
time.sleep(2)

# 총 매장수 확인 및 기타
m_num = driver.page_source
soup = BeautifulSoup(m_num, 'html.parser')
num = int(soup.select_one('#search-result-header > div._54rkq62 > strong').text.replace(',','').replace('개',''))
print(num)
# idx = [262, 283]
idx = 0
# 216]
# id = [114, 155, 178, 180, 182, 185, 189, 192, 198, 204, 212, 213
# 216, 262, 283]
scroll_increment = 130
data = []
# idx = 16

while True:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('.restaurant-info')
    for item in items:
        # current_scroll_position = 0
        if id != 3:    # 스시소라가 계속 안들어가져서 제외하고 크롤링하는 조건문 
            if item is not None:
                time.sleep(3)
                # 가게 리스트 앞부분만 긁어오는 코드
                # Category = item.select_one('.detail .restaurant-meta .tags').text.split(' • ')[0] if item.select_one('.detail .restaurant-meta .tags') else "NA"
                # Location = item.select_one('.detail .restaurant-meta .tags').text.split(' • ')[1] if item.select_one('.detail .restaurant-meta .tags') else "NA"
                # Price_range = item.select_one('.detail .restaurant-meta .price').text if item.select_one('.detail .restaurant-meta .price') else "NA"

                # time.sleep(2)
                # for idx in id:
                current_scroll_position = 0
                # target = driver.find(By.CSS_SELECTOR, f'#virtual_{idx}')
                # Category = target.select_one('.detail .restaurant-meta .tags').text.split(' • ')[0] if target.select_one('.detail .restaurant-meta .tags') else "NA"
                # Location = target.select_one('.detail .restaurant-meta .tags').text.split(' • ')[1] if target.select_one('.detail .restaurant-meta .tags') else "NA"
                # Price_range = target.select_one('.detail .restaurant-meta .price').text if target.select_one('.detail .restaurant-meta .price') else "NA"
            


                targets = driver.find_element(By.CSS_SELECTOR, f'#virtual_{idx}')

                # Parse the HTML of the located element with BeautifulSoup
                target = BeautifulSoup(targets.get_attribute('innerHTML'), 'html.parser')


                category_location = target.select_one('.detail .restaurant-meta .tags').text if target.select_one('.detail .restaurant-meta .tags') else "NA"
                Category = category_location.split(' • ')[0] if ' • ' in category_location else "NA"
                Location = category_location.split(' • ')[1] if ' • ' in category_location else "NA"
                Price_range = target.select_one('.detail .restaurant-meta .price').text if target.select_one('.detail .restaurant-meta .price') else "NA"
                # print(Category, Location, Price_range)

                shop_name = target.select_one('.detail .name').text if target.select_one('.detail .name') else "NA"
                # shop_rating = target.select_one('.detail .restaurant-meta ._1xnbt313 ._1xnbt316 _1ltqxco1k').text if target.select_one('.detail .restaurant-meta ._1xnbt313 ._1xnbt316 _1ltqxco1k') else "NA"
                # r_num = target.select_one('.detail .restaurant-meta ._1xnbt313 ._1xnbt317 _1ltqxco1w').text if target.select_one('.detail .restaurant-meta ._1xnbt313 ._1xnbt317 _1ltqxco1w') else "NA"
                shop_rating = target.select_one('.detail .restaurant-meta ._1xnbt313 ._1xnbt316._1ltqxco1k').text if target.select_one('.detail .restaurant-meta ._1xnbt313 ._1xnbt316._1ltqxco1k') else "NA"
                r_num = target.select_one('.detail .restaurant-meta ._1xnbt313 ._1xnbt317._1ltqxco1w').text if target.select_one('.detail .restaurant-meta ._1xnbt313 ._1xnbt317._1ltqxco1w') else "NA"
                shop_info = target.select_one('.detail .excerpt').text if target.select_one('.detail .excerpt') else "NA"
                # 가게 클릭
                # driver.find_element(By.CSS_SELECTOR, f'#virtual_{idx}').click()
                # time.sleep(2)

                idx += 1

                # 팝업 광고 닫아주는 스크립트
                driver.execute_script("document.querySelectorAll('._1qn1ik44._1ltqxco1u').forEach(a => a.click())")
                time.sleep(1)

                # html = driver.page_source
                # soup = BeautifulSoup(html, 'html.parser')
                


                # # 지도 클릭 및 세부주소
                # driver.execute_script("document.querySelectorAll('._4qv338d.location').forEach(a => a.click())")
                # time.sleep(1)
                # driver.execute_script("document.querySelectorAll('.z8ddia').forEach(a => a.click())")
                # html = driver.page_source
                # soup = BeautifulSoup(html, 'html.parser')
                # shop_name = soup.select_one('.name').text if soup.select_one('.name') else "NA"
                # region = soup.select_one('.addr').text.replace('복사', '') if soup.select_one('.addr') else "NA"
                # time.sleep(3)
                # driver.back()
                # time.sleep(3)

                # print(shop_name)
                data.append([shop_name,shop_rating,r_num,Category,Location,Price_range, shop_info])
                print(shop_name)
                # # 리뷰클릭
                # script = "document.querySelectorAll('._4qv338e').forEach(a => a.click())"
                # driver.execute_script(script)
                # time.sleep(2)
                    
                    #main > section.section.mb-20 > div.section-header._1xvmu764._1xvmu766 > div > div > a

                    # # 페이지 줌 레벨 33%로 설정
                    # driver.execute_script("document.body.style.zoom='33%'")
                    # time.sleep(3)

    

                    # html = driver.page_source
                    # soup = BeautifulSoup(html, 'html.parser')

                    # driver.execute_script("document.querySelectorAll('#main > section.section.mb-20 > div.section-header._1xvmu764._1xvmu765 > div > div._10fm75h2 > a').forEach(a => a.click())")
                    # time.sleep(2)
                    # driver.execute_script("document.querySelectorAll('#main > section.section.mb-20 > div.section-header._1xvmu764._1xvmu765 > div > div._10fm75h2 > div > label:nth-child(2) > span').forEach(a => a.click())")
                    # time.sleep(4)


                    # 리뷰 정렬을 '최근 등록순'으로 설정
                    # set_review_order_to_recent()




                    # # 매장 정보 및 리뷰 가져오기
                    # html = driver.page_source
                    # soup = BeautifulSoup(html, 'html.parser')

                    # driver.execute_script("document.querySelectorAll('#main > section.section.mb-20 > div.section-header._1xvmu764._1xvmu765 > div > div._10fm75h2 > a').forEach(a => a.click())")
                    # time.sleep(2)
                    # driver.execute_script("document.querySelectorAll('#main > section.section.mb-20 > div.section-header._1xvmu764._1xvmu765 > div > div._10fm75h2 > div > label:nth-child(2) > span').forEach(a => a.click())")
                    # time.sleep(4)

                    # # 페이지 줌 레벨 33%로 재설정
                    # driver.execute_script("document.body.style.zoom='50%'")
                    # time.sleep(3)                

                    # shop_rating = soup.select_one('.score').text if soup.select_one('.score') else "NA"
                    # review_num = soup.select_one('#main > div.tab-menu.sticky > ul > li.active > a > span:nth-child(2)').text.replace(',','') if soup.select_one('#main > div.tab-menu.sticky > ul > li.active > a > span:nth-child(2)') else 'NA'
                    
                    # # 리뷰수
                    # r_num = int(review_num)
                    # time.sleep(5)
                    # # 리뷰 백개 뽑기
                    # if r_num > 50:
                    #     for i in range(50):
                    #         html = driver.page_source
                    #         soup = BeautifulSoup(html, 'html.parser')
                            
                    #         # 찜
                    #         save = soup.select_one("#dock > div > span").text if soup.select_one("#dock > div > span") else "NA"
                    #         # 유저 이름
                    #         nickname = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span") else "NA"
                    #         # 유저 별점
                    #         user_rating = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div") else "NA"
                    #         # 점심 저녁
                    #         lun_di = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p") else "NA"
                    #         # 리뷰 등록시간
                    #         times = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span") else "NA"
                    #         # 리뷰
                    #         review = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p") else "NA"

                    #         data.append([shop_name,shop_rating,r_num,Category,Location,region,Price_range,nickname,user_rating,lun_di,times,review,save])
                    #         print(nickname)
                            
                    #         time.sleep(1)
                    #         last_height = driver.execute_script("return document.body.scrollHeight")
                    #         current_scroll_position += scroll_increment
                    #         driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
                    #         time.sleep(2)


                    # else :
                    #     for i in range(r_num):
                    #         html = driver.page_source
                    #         soup = BeautifulSoup(html, 'html.parser')
                            
                    #         # 찜
                    #         save = soup.select_one("#dock > div > span").text if soup.select_one("#dock > div > span") else "NA"
                    #         # 유저 이름
                    #         nickname = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__user-info > a > h4 > span") else "NA"
                    #         # 유저 별점
                    #         user_rating = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div") else "NA"
                    #         # 점심 저녁
                    #         lun_di = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > div > p") else "NA"
                    #         # 리뷰 등록시간
                    #         times = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__header > div.__review-meta.__review-meta--with-rating > span") else "NA"
                    #         # 리뷰
                    #         review = soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p").text if soup.select_one(f"#shopReviewListScroll_{i} > div > article > div.__body > div.__review-post > p") else "NA"
                            
                    #         data.append([shop_name,shop_rating,r_num,Category,Location,region,Price_range,nickname,user_rating,lun_di,times,review,save])
                    #         print(nickname)
                    #         time.sleep(1)
                    #         last_height = driver.execute_script("return document.body.scrollHeight")
                    #         current_scroll_position += scroll_increment
                    #         driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
                    #         time.sleep(2)

                    # 페이지 줌 레벨 100%로 설정
                    # driver.execute_script("document.body.style.zoom='100%'")

                    # time.sleep(3)  # 페이지가 로드될 시간을 줍니다
                    # driver.back()
                    # driver.back()
                    # time.sleep(3)
                
                    # 캐치테이블 가게 리스트 스크롤링
                    # 모달 창 내부를 스크롤 arguments[0].scrollTop += 260 상황에 맞게 px 정해주면 됨
                try:
                    modal_scroll_element = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div > div._53lcbm0._53lcbm1 > div')
                    driver.execute_script('arguments[0].scrollTop += 260', modal_scroll_element)
                    time.sleep(2)
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    items = soup.select('.restaurant-info')
                except StaleElementReferenceException:
                    print("요소를 다시 찾습니다.")
                    modal_scroll_element = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div > div._53lcbm0._53lcbm1 > div')
                    driver.execute_script('arguments[0].scrollTop += 260', modal_scroll_element)
                    time.sleep(2)
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    items = soup.select('.restaurant-info')
        else:
            try:
                modal_scroll_element = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div > div._53lcbm0._53lcbm1 > div')
                driver.execute_script('arguments[0].scrollTop += 260', modal_scroll_element)
                time.sleep(2)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                items = soup.select('.restaurant-info')
            except StaleElementReferenceException:
                print("요소를 다시 찾습니다.")
                modal_scroll_element = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div > div._53lcbm0._53lcbm1 > div')
                driver.execute_script('arguments[0].scrollTop += 260', modal_scroll_element)
                time.sleep(2)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                items = soup.select('.restaurant-info')
            idx += 1
            
            
driver.quit()

584
로우앤슬로우
오스테리아 오르조
이속우화
금토일샴페인빠 한남점
스시우미 용산
쿠촐로 서울
MAISON HANNAM
레에스티우
쇼니노
고청담 용산점
타카 스키야키&우나기
살라댕앰버시
쥬에
한우물
스시츠바사
생면 파스타 전문점 에그앤플라워_해방촌 본점
드포레 와인다이닝 용산
아치서울(arch__seoul) 용산점
초승달
뇨끼바
Cement
내추럴하이
심퍼티쿠시 용산점
쌤쌤쌤
와 서울 (wwah)
심야식당 기억
샤브인서울 신용산 본점
디해방 D HAEBANG
스시 노 아야
미미옥 신용산점 
몽크스부처 이태원
카밀로 한남
휴135
오만지아
ATTA
De tempore 디템포레
NOCT
오스테리아 이아드
사이카쿠
Pimfy 이태원
차경
5412
먼치 Munch
노이 이탈리안 하우스
셰프테이너
소수
알레즈 AL'AISE
뮤땅
스시이젠
TBD 한남
소울 SOUL 
신류 용산점
한쿡 N서울 타워점
신용산 당스
Truffle di Alba (트러플 디 알바)
제이엘 디저트바
춈 (chome)
알아차림 한남점
버터라이스클럽
라쿠치나
한국술집 안씨막걸리
양식문화
마제 스키야키
쎄오
ZERO COMPLEX
부다스벨리
판코네(Pancone)
라인드 한남
CESTA 세스타
파스티피쵸
치즈플로
피에세(PIECE)
방방
어물전 청 한남점
이속우화 덴푸라 혼 
목계화원
uphill namsan
살팀보카
로기
야스노야 본점
화양연화
소와나 한남 본점
한남타츠
레이지파머스
스시사이
Barboo (바부)
스시고코로
미래회관 한남
사테
스시쵸우
제이든by한남
스이세한남
DOTZ 
비아톨레도 파스타바
에프더블유브이 용산점
스시상남
Trattoria & Bar VILLA DE LA VINO
WUI 웨이 티하우스 & 레스토랑
개나리식당
기러기둥지 용산 'The Train'
엔그릴 (n GRILL)
Restaurant I.O.U
pakin
꼼모아
사랑이 뭐길래
KEEM
더즌 오이스터 한남
보르고한남
몬드리안 클레오(CLEO)
마음집
MAISON HANNAM V
르다 (Le Da)
라페름
Harbou

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#virtual_330"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6361FEEA2+31554]
	(No symbol) [0x00007FF636177ED9]
	(No symbol) [0x00007FF63603872A]
	(No symbol) [0x00007FF636088434]
	(No symbol) [0x00007FF63608853C]
	(No symbol) [0x00007FF6360CF6A7]
	(No symbol) [0x00007FF6360AD06F]
	(No symbol) [0x00007FF6360CC977]
	(No symbol) [0x00007FF6360ACDD3]
	(No symbol) [0x00007FF63607A33B]
	(No symbol) [0x00007FF63607AED1]
	GetHandleVerifier [0x00007FF636508B1D+3217341]
	GetHandleVerifier [0x00007FF636555AE3+3532675]
	GetHandleVerifier [0x00007FF63654B0E0+3489152]
	GetHandleVerifier [0x00007FF6362AE776+750614]
	(No symbol) [0x00007FF63618375F]
	(No symbol) [0x00007FF63617EB14]
	(No symbol) [0x00007FF63617E

In [10]:
df = pd.DataFrame(data)
df.to_excel('c.xlsx')